# SWE 546 Spring 2016 Assignment 4

Creating an algorithm for predicting Movie ratings for users using Stochastic Gradient Descent on Movielens 1M dataset.

## Loading Data

In [2]:
import numpy as np
import pandas as pd
from sklearn import linear_model

moviesF = 'data/movies.dat'
ratingsF = 'data/ratings.dat'
usersF = 'data/users.dat'


moviesC = ['movie_id', 'title', 'genres']
moviesD = pd.read_table(moviesF, sep='::', header = None, names = moviesC,engine='python')

usersC = ['user_id', 'gender', 'age', 'occupation', 'zip']
usersD  = pd.read_table(usersF, sep='::', header = None, names = usersC,engine='python')

ratingsC  = ['user_id', 'movie_id', 'rating', 'timestamp']
ratingsD = pd.read_table(ratingsF, sep='::', header = None, names = ratingsC,engine='python')


#pivotData = pd.merge(pd.merge(ratingsD, usersD), moviesD)

#mean_ratings = pivotData.pivot_table('ratings', index = 'title', columns = 'gender', aggfunc = 'mean' )

ratingsD = ratingsD.drop('timestamp',1)
print(ratingsD[:5])

ratingCount = len(ratingsD)

#print(ratingsD[:,2:5])



   user_id  movie_id  rating
0        1      1193       5
1        1       661       3
2        1       914       3
3        1      3408       4
4        1      2355       5


## Getting Movie and User Count

In [3]:
### 
maxMovieId = ratingsD['movie_id'].max()  
maxUserId = ratingsD['user_id'].max() 

print("Maximum Movie ID = ",maxMovieId)
print("Maximum User ID = ",maxUserId)

Maximum Movie ID =  3952
Maximum User ID =  6040


## Creating Filter Matrix
### Creating Zeroes

In [4]:
shape = (maxUserId+1,maxMovieId+1)
Y = np.zeros(shape)
M = Y

Ys = np.hstack([np.matrix(ratingsD['rating']).T,np.matrix(ratingsD['user_id']).T,np.matrix(ratingsD['movie_id']).T])

print(Ys.shape[0])

1000209


### Marking available data

In [5]:
for x in range (ratingCount):
    userId = ratingsD['user_id'][x]
    movieId = ratingsD['movie_id'][x]
    ratingValue = ratingsD['rating'][x]
    Y[userId,movieId] = ratingValue
    M[userId,movieId] = 1
print(Y)
print(M)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


In [10]:
A = np.mat(np.random.rand(maxUserId+1, 3))
B = np.mat(np.random.rand(3, maxMovieId+1))

print(A)
print(B)


[[ 0.62084375  0.63397294  0.30756991]
 [ 0.97637795  0.12129708  0.73865627]
 [ 0.76608543  0.52600338  0.48305735]
 ..., 
 [ 0.22784055  0.8928121   0.35209273]
 [ 0.18567566  0.01974746  0.65325415]
 [ 0.80612824  0.08751519  0.41768916]]
[[ 0.13625256  0.51998585  0.27482045 ...,  0.13871741  0.57324093
   0.67396946]
 [ 0.66046245  0.29084605  0.28097574 ...,  0.33553806  0.63059776
   0.13325599]
 [ 0.85279009  0.40135268  0.62959464 ...,  0.71091726  0.13902145
   0.32102871]]


## Gradient Descent Batch Size 3

In [11]:
EPOCH = 100
Eta = 0.1
eta = Eta

for i in range(EPOCH):
    for k in range(Y.shape[0]):
        u = Ys[k,1]
        m = Ys[k,2]
        err = Ys[k,0] - A[u,:]*B[:,m]
        temp_A = A[u,:] + eta*err[0,0]*B[:,m].T
        B[:,m]= B[:,m] + eta*err[0,0]*A[u,:].T
        A[u,:]= temp_A
    
    eta = Eta*1./(i+1)

In [12]:
E = np.array(M)*np.array(Y - A*B)
np.sum(E*E)/np.sum(np.array(M))
np.sum(np.array(M))

1000209.0

In [13]:
res = A*B
for j in range (20):
    
    userResult = [(0,0)]
    for m in range (maxMovieId+1):
        movieTuple = (res[j,m],m)
        userResult.append(movieTuple)
        
    userResult = sorted(userResult, key=lambda tup: tup[0], reverse=True)     
    
    print("Movie for user " , j , " : ")
    for n in range(10):
        print (userResult[n])

Movie for user  0  : 
(2.004020366866095, 3083)
(1.9705210366031851, 3405)
(1.9575509885177826, 2262)
(1.9372669168258823, 3871)
(1.9287048278978252, 2337)
(1.9260316199540966, 2917)
(1.896523875042347, 771)
(1.865442165775405, 1333)
(1.8627491280400217, 2920)
(1.8564474708707042, 3163)
Movie for user  1  : 
(6.5819048003824481, 3083)
(6.2413641603456078, 2917)
(6.1230193355782667, 3943)
(6.0954970331513465, 2262)
(5.9874269634233492, 1176)
(5.9314618459626232, 3871)
(5.9288642710323032, 2699)
(5.8891926605258931, 771)
(5.8733308206631483, 2337)
(5.8390492501724944, 1237)
Movie for user  2  : 
(6.1745022153904614, 2699)
(6.0453850436730558, 3943)
(6.0019856086923644, 3083)
(5.9147775899223758, 2262)
(5.8530836231526999, 1237)
(5.8359061325315329, 1176)
(5.7400504848516247, 2447)
(5.6303181037741101, 3543)
(5.5927672896319676, 2920)
(5.5825261521645846, 3871)
Movie for user  3  : 
(6.2461525214014815, 3083)
(6.0345723604185952, 2262)
(5.9874899949721021, 3943)
(5.8735185776700956, 1176)